Amadéo David (44761700) - Marine Van Renterghem (31621700) - Sylvie Van Schendel (44841700)

# LINMA1702 Optimisation d'un réseau de distribution d'eau

## I. Analyse d'un réseau existant

### I.1 Que représentent les expressions $A^Th$ et $Af$ ?

Soit h le vecteur contenant les hauteurs en chaque point, f le vecteur contenant les débits circulant dans les conduites et A la matrice d'incidence.

L'expression $A^Th$ est le vecteur contenant les dénivelés de chaque conduite.

L'expression $Af$ est le vecteur contenant les débits nets en chaque point.

### I.2 

### I.4 Résolution numérique du problème I.3 avec les données fournies

In [1]:
from scipy.optimize import linprog # import du solveur d'optimisation linéaire à utiliser
import matplotlib.pyplot as plt # pour graphiques éventuels
import numpy as np

In [20]:
#Fonction retournant [f,xC,xA] avec f la valeur de la fonction objectif maximale 
#xC un vecteur des valeurs optimales des débits aux points de consommation
#xA un vecteur des valeurs optimales des débits aux points d'approvisionnement
#Soit Pf le prix facturé
#Soit C le cout
#Soit DCmin et DCmax les valeurs minimales et maximales du débit aux points de consommation
#Soit DAmax les valeurs maximales du débit extractibles aux sources
def Analyse(nPoints,nConduites,X,Y,Z,A,R,Pa,Damax,C,Pc,Dcmin,Dcmax,Pf,Pi,alpha):
    """
    Analyse renvoie un vecteur retournant, dans l'ordre :
        ° la valeur de la fonction objectif maximale;
        ° un vecteur des valeurs optimales des débits aux points de consommation;
        ° un vecteur des valeurs optimales des débits aux points d'approvisionnement.
        
    """
    L=np.zeros(nConduites)
    for i in range(nConduites): 
        p=0
        P1=np.zeros(3)
        P2=np.zeros(3)
        while(A[p][i]==0): 
            p+=1
        P1[0]=X[p]
        P1[1]=Y[p]
        P1[2]=Z[p]
        for j in range(p+1,nPoints): 
            if A[j][i]==-1 or A[j][i]==1 : 
                P2[0]=X[j]
                P2[1]=Y[j]
                P2[2]=Z[j]
                L[i]+=np.sqrt((P2[0]-P1[0])**2+(P2[1]-P1[1])**2+(P2[2]-P1[2])**2)
                P1=P2.copy()
                P2=np.zeros(3)                
    h=A.T@Z
    print(A[Pc])#Pourquoi ça fonctionne pas?
    print(A[Pa])#Pourquoi ça fonctionne pas?
    A1=np.concatenate((A[Pc],A[Pa]))
    Debmax=np.multiply(np.outer(np.ones(len(A1)),alpha*R*R*h/L),A1)
    A2=A[Pc]
    Debmin=-1*np.multiply(np.outer(np.ones(len(A2)),alpha*R*R*h/L),A2)
    Sum=np.sum(np.multiply(np.outer(np.ones(len(A1)),alpha*R*R*h/L),A1),axis=0)
    Aub=np.concatenate((np.concatenate((Debmax,Debmin)),Sum))
    c=np.multiply(np.multiply(np.outer(np.ones(len(A1)),alpha*R*R*h/L),A1).T,np.concatenate((Pf,C))).T
    #c  = -1* alpha*h*R**2/L * np.concatenate((Pf, -1*np.array(C)))       # On traite un problème de maximisation
    #A1 = np.eye(len(Dcmin)+len(Damax))
    #A2 = -1 * np.eye(len(Dcmin))                         # Matrice identite
    #A2Add = np.zeros((len(Dcmin),len(Damax)))            # Matrice de 0
    #A2 = np.concatenate((A2,A2Add),axis=1)               # Ajout de 0 a la fin de la matrice
    #A5  = np.concatenate((A1,A2))
    #A3 = np.ones((1,len(Dcmin)+len(Damax)))
    #A3[0,len(Dcmin):(len(Dcmin)+len(Damax))]*=-1
    #A6  = alpha*h*R**2/L*np.concatenate((A5,A3))
    b  = np.concatenate((np.concatenate((Dcmax,-1*np.array(Damax))),np.concatenate((-1*np.array(Dcmin),[0]))))
    res= linprog(c, A_ub = Aub, b_ub = b)#Modifier les bounds
    print(c)
    return [-1*res.fun, res.x[0:len(Dcmin)],res.x[len(Dcmin):len(Dcmin)+len(Damax)]]#modifier le return

In [21]:
def readFile(fileName): 
    with open(fileName,"r") as f :
        nPoints = int(f.readline().split()[0])
        nConduites=int(f.readline().split()[0])
        f.readline()
        f.readline()
        xyz   = np.array(list(list(float(w) for w in f.readline().split()[:]) for i in range(nPoints)))
        f.readline()
        f.readline()
        A = np.array(list(list(float(w) for w in f.readline().split()[:]) for i in range(nPoints)))
        alpha=10**6
        R=1
        Pa=[2, 6]
        Damax=[10000, 5000]
        C= [0.2, 0.3]
        Pc=[1, 4, 10, 20, 23]
        Dcmin=[200, 200, 200, 200, 200]
        Dcmax=[4000, 2000, 2000, 3000, 2000]
        Pf=0.9
        Pi=[3, 5, 7, 8, 9, 11, 12 ,13, 14, 15, 16, 17, 18, 19, 21, 22]
        X = xyz[:,0]
        Y = xyz[:,1]
        Z = xyz[:,2]
        
    return [nPoints,nConduites,X,Y,Z,A,R,Pa,Damax,C,Pc,Dcmin,Dcmax,Pf,Pi,alpha]
    

In [22]:
[nPoints,nConduites,X,Y,Z,A,R,Pa,Damax,C,Pc,Dcmin,Dcmax,Pf,Pi,alpha]=readFile("Mini-exemple-avec-solution/Data-mini-exemple-avec-solution.txt")
print(Analyse(nPoints,nConduites,X,Y,Z,A,R,Pa,Damax,C,Pc,Dcmin,Dcmax,Pf*np.ones(len(Dcmin)),Pi,alpha))

IndexError: index 23 is out of bounds for axis 0 with size 23

# II. Améliorations du réseau

# III. Conception d'un réseau optimal